In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [ ]:
df = pd.read_csv('dataset.csv')
df

In [ ]:
df[['SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude']].drop_duplicates()

In [ ]:
df['SystemCodeNumber'].nunique()

In [ ]:
df.groupby('SystemCodeNumber')[['Capacity','Occupancy','QueueLength']].agg(['min','max','mean'])

Maximum occupancy is more than capacity in some cases which may cause a problem

In [ ]:
demand_fluctuation=((df.groupby('SystemCodeNumber')['Occupancy'].max()-df.groupby('SystemCodeNumber')['Occupancy'].min())/df.groupby('SystemCodeNumber')['Capacity'].max())
demand_fluctuation

Since the (demand fluctuation)/capacity of each parking space is less than 1, the weight (alpha) for it can be set to 1.

In [ ]:
TRAFFIC_ENCODING = {'low': 0, 'average': 0.5, 'high': 1}
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].apply(lambda x: TRAFFIC_ENCODING.get(x))

In [ ]:
VEHICLE_TYPE_WEIGHT = {'cycle': 0.4, 'bike': 0.7, 'car': 1, 'truck': 1.5}
df['VehicleType'] = df['VehicleType'].apply(lambda x: VEHICLE_TYPE_WEIGHT.get(x, 1))

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
df[['Timestamp', 'SystemCodeNumber' , 'Capacity', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay']].to_csv("parking_stream.csv", index=False)

In [ ]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Capacity: int
    Occupancy: int
    VehicleType: float
    TrafficConditionNearby: float
    QueueLength: int
    IsSpecialDay: int

In [ ]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# Model 1: Baseline Model

In [ ]:
import datetime
BASE_PRICE = 10
alpha = 1.0
@pw.udf
def compute_price_model1(occupancy, capacity):
    if capacity > 0:
        occ_rate = occupancy / capacity
    else:
        occ_rate = 0
    price = BASE_PRICE + alpha * occ_rate * BASE_PRICE
    return max(5, min(price, 20))

delta_window_model1 = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior(),
    )
    .reduce(
        t=pw.this._pw_window_end,
        lot_id=pw.reducers.any(pw.this.SystemCodeNumber),
        occ_max=pw.reducers.max(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
    )
    .with_columns(
        price=compute_price_model1(
            occupancy=pw.this.occ_max,
            capacity=pw.this.cap
            )
    ))


# Model 2: Demand-Based Price Function


In [ ]:
import pathway as pw
import datetime

BASE_PRICE = 10
weight_occupancy = 1.0
weight_queue = 0.5
weight_traffic = 0.7
weight_special = 0.5
weight_vehicle = 0.8

def compute_demand(occupancy, capacity, queue, traffic, special, vtype):
    if capacity > 0:
        occ_rate = occupancy / capacity
    else:
        occ_rate = 0
    demand = (
        weight_occupancy * occ_rate
        + weight_queue * queue
        - weight_traffic * traffic
        + weight_special * special
        + weight_vehicle * vtype
    )
    return demand

def normalize_demand(demand, min_d=0, max_d=5):
    demand = max(min(demand, max_d), min_d)
    return (demand - min_d) / (max_d - min_d)

@pw.udf
def compute_price_model2(occupancy, capacity, queue, traffic, special, vtype):
    demand = compute_demand(occupancy, capacity, queue, traffic, special, vtype)
    norm_d = normalize_demand(demand)
    price = BASE_PRICE * (1 + norm_d)
    return max(5, min(price, 20))

delta_window_model2 = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior(),
    )
    .reduce(
        t=pw.this._pw_window_end,
        lot_id=pw.reducers.any(pw.this.SystemCodeNumber),
        occ=pw.reducers.max(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        queue=pw.reducers.max(pw.this.QueueLength),
        traffic=pw.reducers.max(pw.this.TrafficConditionNearby),
        special=pw.reducers.max(pw.this.IsSpecialDay),
        vtype=pw.reducers.max(pw.this.VehicleType),
    )
    .with_columns(
        price=compute_price_model2(
            pw.this.occ,
            pw.this.cap,
            pw.this.queue,
            pw.this.traffic,
            pw.this.special,
            pw.this.vtype,
        )
    )
)

# Visualising Using Bokeh

In [ ]:
from bokeh.palettes import Category20

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Daily Parking Price",
        x_axis_type="datetime",
        x_axis_label="Time",
        y_axis_label="Price ($)"
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig


In [ ]:
# Use Pathway's built-in visualization module to visualize the tables
viz = delta_window_model1.plot(price_plotter, sorting_col="t")
viz2= delta_window_model2.plot(price_plotter, sorting_col="t")
# Create a Panel layout and make it servable as a web app
pn.Column(viz,viz2).servable()

In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()